# Pixie dust and spark: Visualizaitons

In [1]:
!pip install --upgrade pixiedust
#duh...

    100% |████████████████████████████████| 163kB 4.4MB/s eta 0:00:01
    100% |████████████████████████████████| 798kB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 5.5MB 219kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 6.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for pixiedust ... done
  Stored in directory: /gpfs/fs01/user/sa0d-218294071c3031-cf228a6161b9/.cache/pip/wheels/8f/54/78/7c182cc388e19b8077f4ba76b2777d503f09a86194ca9d6fce
  Running setup.py bdist_wheel for mpld3 ... done
  Stored in directory: /gpfs/fs01/user/sa0d-218294071c3031-cf228a6161b9/.cache/pip/wheels/69/bc/68/7ca3b696749d183e998968fc24b0ff3c5e119d9e68bf495b07
Successfully built pixiedust mpld3


In [1]:
import pixiedust
# and then import

Pixiedust database opened successfully


In [2]:
inspections = pixiedust.sampleData("https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv")
#Load data from the said URL
# CSV 

Downloaded 364310 bytes
Creating pySpark DataFrame for 'https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://opendata.lasvegasnevada.gov/resource/86jg-3buh.csv'


In [3]:
#view data using the dislay(dataFrameName)
display(inspections)

:@computed_region_tnyv_z3b7,address,category_name,city,current_demerits,current_grade,date_current,employee_id,inspection_date,inspection_demerits,inspection_grade,inspection_result,inspection_time,inspection_type,location_1,location_1_address,location_1_city,location_1_state,location_1_zip,location_name,permit_number,permit_status,record_updated,restaurant_name,serial_number,state,violations,zip
None,4000 W Flamingo Rd,Bar / Tavern,Las Vegas,5,A,2017-08-07 00:00:00,EE7001214,2017-08-07 00:00:00,5,A,'A' Grade,2017-08-07 15:20:00,Routine Inspection,POINT (115.1929046 36.1166353),None,None,None,None,Gold Coast Hotel & Casino,PR0006774,A,2017-08-08 09:31:16,Gold Coast Bingo Service Bar,DA0EHXVCT,Nevada,209,89103-4004
None,1237 Nevada Hwy,Restaurant,Boulder City,5,A,2017-08-08 00:00:00,EE7001020,2017-08-08 00:00:00,5,A,'A' Grade,2017-08-08 08:00:00,Routine Inspection,POINT (114.8491471 35.9716148),None,None,None,None,McDonald's Restaurant,PR0002776,A,2017-08-08 08:52:21,McDonald's Restaurant #7790,DA0J0HTDI,Nevada,"202,2927",89005-1809
None,1592 N Eastern Ave,Snack Bar,Las Vegas,6,A,2017-08-07 00:00:00,EE7001175,2017-08-07 00:00:00,6,A,'A' Grade,2017-08-07 14:35:00,Routine Inspection,POINT (115.1156331 36.188224),None,None,None,None,7-Eleven Food Store #22293 C,PR0003263,A,2017-08-07 15:09:39,7-Eleven #22293 - Deli,DA0WQ53CA,Nevada,"215,2927,2955",89101-1509
None,4000 W Flamingo Rd,Bar / Tavern,Las Vegas,3,A,2017-08-07 00:00:00,EE7001214,2017-08-07 00:00:00,3,A,'A' Grade,2017-08-07 13:20:00,Routine Inspection,POINT (115.1929046 36.1166353),None,None,None,None,Gold Coast Hotel & Casino,PR0006764,A,2017-08-07 16:51:28,Gold Coast Island Bar,DA16C7JRS,Nevada,215,89103-4004
None,5485 W Flamingo Rd,Snack Bar,Las Vegas,6,A,2017-07-24 00:00:00,EE7000674,2017-07-24 00:00:00,6,A,'A' Grade,2017-07-24 09:20:00,Routine Inspection,POINT (115.216013 36.114652),None,None,None,None,SPRING VALLEY MARKET,PR0099163,A,2017-08-08 10:24:52,SPRING VALLEY MARKET SNACK BAR,DA1800421,Nevada,"212,218",89103-3500
None,3700 W Flamingo Rd,Kitchen Bakery,Las Vegas,3,A,2017-08-03 00:00:00,EE7000674,2017-08-03 00:00:00,3,A,'A' Grade,2017-08-03 12:45:00,Routine Inspection,POINT (115.1848942 36.1164467),None,None,None,None,Rio Suites Hotel,PR0005137,A,2017-08-08 10:37:21,Rio Bakery,DA1800433,Nevada,"216,2930",89103-4043
None,3700 W Flamingo Rd,Restaurant,Las Vegas,8,A,2017-08-03 00:00:00,EE7000674,2017-08-03 00:00:00,8,A,'A' Grade,2017-08-03 14:05:00,Routine Inspection,POINT (115.1848942 36.1164467),None,None,None,None,Rio Suites Hotel,PR0005161,A,2017-08-08 11:10:29,Rio Employee Dining Room Bistro,DA1800443,Nevada,"209,218,2930",89103-4043
None,2121 E Craig Rd,Bar / Tavern,North Las Vegas,0,A,2017-08-04 00:00:00,EE7000390,2017-08-04 00:00:00,0,A,'A' Grade,2017-08-04 14:00:00,Routine Inspection,POINT (115.1190876 36.23855),None,None,None,None,Cannery Hotel & Casino,PR0012863,A,2017-08-08 11:13:49,CANNERY WAVERLY'S - BAR,DA1800444,Nevada,None,89030-3385
None,2121 E Craig Rd,Bar / Tavern,North Las Vegas,3,A,2017-08-04 00:00:00,EE7000390,2017-08-04 00:00:00,3,A,'A' Grade,2017-08-04 13:20:00,Routine Inspection,POINT (115.1190876 36.23855),None,None,None,None,Cannery Hotel & Casino,PR0012865,A,2017-08-08 11:17:01,CANNERY CASA COCINA - BAR,DA1800445,Nevada,2955,89030-3385
None,2121 E Craig Rd,Bar / Tavern,North Las Vegas,3,A,2017-08-04 00:00:00,EE7000390,2017-08-04 00:00:00,3,A,'A' Grade,2017-08-04 12:15:00,Routine Inspection,POINT (115.1190876 36.23855),None,None,None,None,Cannery Hotel & Casino,PR0012866,A,2017-08-08 11:24:20,CANNERY Service Bar #2 'Club',DA1800448,Nevada,"2928,2930,2955",89030-3385


In [4]:
inspections.registerTempTable("restaurants")
#register inspections dataframe as a table named "restaurants"
lasDF = sqlContext.sql("SELECT * FROM restaurants WHERE city='Las Vegas'")
lasDF.count()
#lasDF is a dataframe that has restaurants with city as Las Vegas

832

In [8]:
!pip install --user --upgrade bokeh
# tried using bokeh as renderer 

# Kernel needs to be restarted after this installation of bokeh

    100% |████████████████████████████████| 15.4MB 78kB/s  eta 0:00:01    94% |██████████████████████████████▎ | 14.5MB 52.3MB/s eta 0:00:01
Requirement already up-to-date: six>=1.5.2 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa0d-218294071c3031-cf228a6161b9/.local/lib/python3.5/site-packages (from bokeh)
    100% |████████████████████████████████| 256kB 4.2MB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 4.8MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 5.1MB/s eta 0:00:01
    100% |████████████████████████████████| 17.1MB 72kB/s  eta 0:00:01    67% |█████████████████████▌          | 11.5MB 56.1MB/s eta 0:00:01
    100% |████████████████████████████████| 491kB 2.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for bokeh ... done
  Stored in directory: /gpfs/fs01/user/sa0d-218294071c3031-cf228a6161b9/.cache/pip/wheels/a7/2c/4a/96740179eabf7ddba1f7ae36ba96540e8a7e557936eaab4829
  Running setup.py bdist_wheel for PyYAML ... done
  Stored in di

In [5]:
bycat = lasDF.groupBy("category_name").count()
display(bycat)
#group by category

### From the drop down tol bar above: choose bar chart option
#### select Key: X value category, and value: values plotted against Y axis...
#### chart ala.. pixie dust

In [6]:
display(lasDF)

In [7]:
display(lasDF)
# another display

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *


def valueToLon(value):
    lon = float(value.split('POINT (')[1].strip(')').split(' ')[0]) 
    return None if lon == 0 else lon if lon < 0 else (lon * -1) # Data cleaning 
#if lon <0 {do nothing} else{multiply lon by -1 and return that value}
# the longitudes were supposed to be negative but none of the input values had the negative sign

def valueToLat(value):
    lat = float(value.split('POINT (')[1].strip(')').split(' ')[1])
    return None if lat == 0 else lat # Data cleaning

udfValueToLon = udf(valueToLon, DoubleType()) #user defined funciton DoubleType :: return type
udfValueToLat = udf(valueToLat, DoubleType())

lonDF = lasDF.withColumn("lon", udfValueToLon("location_1"))
lonlatDF = lonDF.withColumn("lat", udfValueToLat("location_1")) # the prev DF is being used in this line

lonlatDF.printSchema() #hence has both lat and lon


root
 |-- :@computed_region_tnyv_z3b7: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- current_demerits: integer (nullable = true)
 |-- current_grade: string (nullable = true)
 |-- date_current: timestamp (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- inspection_date: timestamp (nullable = true)
 |-- inspection_demerits: integer (nullable = true)
 |-- inspection_grade: string (nullable = true)
 |-- inspection_result: string (nullable = true)
 |-- inspection_time: timestamp (nullable = true)
 |-- inspection_type: string (nullable = true)
 |-- location_1: string (nullable = true)
 |-- location_1_address: string (nullable = true)
 |-- location_1_city: string (nullable = true)
 |-- location_1_state: string (nullable = true)
 |-- location_1_zip: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- permit_number: string (nullable = true)
 |-- permit_

In [13]:
display(lonlatDF)

# And lo behold
## use the map layout on pixie dust..
### use th ekey generated from the mapbox account
